## CNN을 이용한 컬러 이미지 분류

### Step 1. Task 설정
동물, 기계 등 여러가지 아이템의 이미지를 담고 있는 CIFAR-10 데이터셋을 활용해서 분류 Task를 수행하고자 한다.   
CIFAR-10 데이터셋의 출력 라벨은 10개로, 이는 지도학습의 다중분류 Task가 되겠다.   
해당 Task를 위해서 이미지 분류에 탁월한 성능을 보이는 CNN을 활용하고자 한다.   
**단, 이번에는 Step2인 데이터 전처리 과정을 추가하여 모델의 성능을 높여보고자 한다.

In [1]:
# 필요한 모듈을 불러온다.
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout,\
GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

### Step 2. 데이터셋 준비하기
#### 데이터셋 살펴보기: CIFAR-10 데이터셋
- CIFAR-10은 자동차/비행기/강아지/고양이 등 여러가지 이미지들을 모아둔 데이터셋
- 이미지: 각 32*32픽셀, 컬러 이미지
- 샘플수: 총 60,000
- 총 10가지의 class가 있음(클래스별 6천개)

In [2]:
# 마찬가지로 CIFAR-10으로 분석한다
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train/255.0, x_test/255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
print('x_train.shape:', x_train.shape)
print('y_train.shape:', y_train.shape)

x_train.shape: (50000, 32, 32, 3)
y_train.shape: (50000,)


In [3]:
# number of classes
K = len(set(y_train))
print('number of classes:', K)

number of classes: 10


### 데이터 정규화 Batch Normalization
- 분석속도 up
- 정확도 up 


데이터 정규화를 하는 이유:
- 학습속도 up
- 가중치 초깃값 선택 의존성 감소
- 과적합 위험 감소
- 기울기 소실 문제 해결
- Dropout 대체 가능.

데이터 정규화 = 활성함수의 '출력값'을 정규화 해준다.

In [4]:
# 입력층
i = Input(shape=x_train[0].shape)

# 이전에 실행했던 CNN 코드
# x = Conv2D(32, (3,3), strides=2, activation='relu')(i)
# x = Conv2D(64, (3,3), strides=2, activation='relu')(x)
# strides 옵션은 학습속도를 줄이기 위한 목적

# Stage 1.

# 1층
x = Conv2D(32, (3,3), activation='relu')(i)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

# 2층
x = Conv2D(32, (3,3), activation='relu')(i)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

# 3층
x = Conv2D(32, (3,3), activation='relu')(i)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

In [5]:
x = Flatten()(x)
x = Dropout(0.2)(x)

In [6]:
# Stage 2. ANN

x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(K, activation='softmax')(x)

model = Model(i,x)

In [7]:
model.compile(optimizer='adam',
              loss = 'sparse_categorical_crossentropy',
              metrics =['accuracy'])

In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5939852525939697147
xla_global_id: -1
]


In [10]:
r = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=15)

Epoch 1/15
  22/1563 ━━━━━━━━━━━━━━━━━━━━ 4:40 182ms/step - accuracy: 0.6374 - loss: 1.0046

KeyboardInterrupt: 

### Data augmentation 적용하기

In [ ]:
batch_size = 32
data_generator = \
tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1,
                                                height_shift_range=0.1,
                                                horizontal_flip=True)
train_generator = data_generator.flow(x_train, y_train, batch_size)

steps_per_epoch = x_train.shape[0] // batch_size
r = model.fit(train_generator, validation_data=(x_test, y_test), \
              steps_per_epoch=steps_per_epoch, epochs=50)